In [2]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import os

## Save best training metrics

In [3]:
with open(r'Z:\grodriguez\CardiacOCT\data-2d\results\nnUNet\2d\Task501_CardiacOCT\nnUNetTrainerV2__nnUNetPlansv2.1\cv_niftis_postprocessed\summary.json') as f:
    summary = json.load(f)

In [4]:
#Merge frames into pullbacks

pullbacks_origs = os.listdir(r'Z:\grodriguez\CardiacOCT\data-2d\results\nnUNet\2d\Task501_CardiacOCT\nnUNetTrainerV2__nnUNetPlansv2.1\cv_niftis_postprocessed')
pullbacks_origs_set = []
pullbacks_dict = {}

for i in range(len(pullbacks_origs)):
    if pullbacks_origs[i].split('_frame')[0] not in pullbacks_origs_set:
        pullbacks_origs_set.append(pullbacks_origs[i].split('_frame')[0])

    else:
        continue

for i in range(len(pullbacks_origs_set)):
    frames_from_pullback = [frame for frame in pullbacks_origs if pullbacks_origs_set[i] in frame]
    pullbacks_dict[pullbacks_origs_set[i]] = frames_from_pullback

#Remove last key-value pair
keys = list(pullbacks_dict.keys())
last_key = keys[-1]
pullbacks_dict[last_key].pop()
if not pullbacks_dict[last_key]:
    pullbacks_dict.pop(last_key)

In [5]:
pullbacks_dict

{'NLDAMPH0067_1': ['NLDAMPH0067_1_frame280_019.nii.gz',
  'NLDAMPH0067_1_frame80_019.nii.gz',
  'NLDAMPH0067_1_frame320_019.nii.gz',
  'NLDAMPH0067_1_frame400_019.nii.gz',
  'NLDAMPH0067_1_frame120_019.nii.gz',
  'NLDAMPH0067_1_frame200_019.nii.gz',
  'NLDAMPH0067_1_frame40_019.nii.gz',
  'NLDAMPH0067_1_frame160_019.nii.gz',
  'NLDAMPH0067_1_frame0_019.nii.gz',
  'NLDAMPH0067_1_frame240_019.nii.gz'],
 'NLDAMPH0063_1': ['NLDAMPH0063_1_frame300_016.nii.gz',
  'NLDAMPH0063_1_frame520_016.nii.gz',
  'NLDAMPH0063_1_frame200_016.nii.gz',
  'NLDAMPH0063_1_frame400_016.nii.gz',
  'NLDAMPH0063_1_frame40_016.nii.gz',
  'NLDAMPH0063_1_frame320_016.nii.gz',
  'NLDAMPH0063_1_frame120_016.nii.gz',
  'NLDAMPH0063_1_frame307_016.nii.gz',
  'NLDAMPH0063_1_frame297_016.nii.gz',
  'NLDAMPH0063_1_frame522_016.nii.gz',
  'NLDAMPH0063_1_frame240_016.nii.gz',
  'NLDAMPH0063_1_frame160_016.nii.gz',
  'NLDAMPH0063_1_frame440_016.nii.gz',
  'NLDAMPH0063_1_frame286_016.nii.gz',
  'NLDAMPH0063_1_frame290_016.nii.

In [87]:
def merge_dicts(list_dicts):

    result = {}
    for d in list_dicts:
        for label, metrics in d.items():
            if label not in result:
                if np.isnan(metrics['Dice']):
                    metrics['Dice'] = 0.0
                result[label] = [metrics['Dice']]

            else:
                if np.isnan(metrics['Dice']):
                    metrics['Dice'] = 0.0
                result[label].append(metrics['Dice'])
            
    return result
    

In [88]:
def mean_metrics(dict_pullback):
    
    for label, dices in dict_pullback.items():
        dict_pullback[label] = np.mean(dices)

    return dict_pullback

In [93]:
list_dicts_pullback = []
final_dict = {}

for pullback in pullbacks_dict:

    for frame in pullbacks_dict[pullback]:

        for sub_dict in summary['results']['all']:

            if sub_dict['reference'] == '/mnt/netcache/diag/grodriguez/CardiacOCT/data-2d/results/nnUNet/2d/Task501_CardiacOCT/nnUNetTrainerV2__nnUNetPlansv2.1/gt_niftis/{}'.format(frame):
                list_dicts_pullback.append({k: v for i, (k, v) in enumerate(sub_dict.items()) if i < len(sub_dict) - 2})

            else:
                continue

    result = merge_dicts(list_dicts_pullback)
    mean_result = mean_metrics(result)
    final_dict[pullback] = mean_result

In [95]:
with open(r'Z:\grodriguez\CardiacOCT\metrics_build\pullback_metrics.json', 'w') as f:
    json.dump(final_dict, f, indent=4)

## Create Excel file

### First sheet

In [ ]:
overview = {'Date': '14/02/2023',
        'Nº patients': 62,
        'Nº pullbacks': 70,
        'Nº frames with annots': 783,
        'Model': 'Model2d_1',
        'Runtime': '~27h per fold',
        'Avg performance': }

### Second sheet (best performance on 1st dataset)

In [96]:
with open(r'Z:\grodriguez\CardiacOCT\metrics_build\pullback_metrics.json', 'r') as f:
    data = pd.read_json(f, orient='index')

In [109]:
data.rename(columns={
        1: "lumen",
        10: "wthrombus",
        11: "dissection",
        12: "rupture",
        2: "guidewire",
        3: "wall",
        4: "lipid",
        5: "calcium",
        6: "media",
        7: "catheter",
        8: "sidebranch",
        9: "rthrombus"}, inplace=True)

In [113]:
data.to_csv('./metrics.csv')

### Third sheet (best performance on 2nd dataset)

### Fourth sheet (performance of Model 1 and 2 on test set)